In [26]:
import requests
import json
import pandas as pd
from datetime import datetime
import concurrent.futures
import time
import logging
import os

class DeribitHTTP:
    def __init__(self, logger=None):
        self.url = "https://www.deribit.com/api/v2/"
        self.logger = logger or logging.getLogger(__name__)

    def __send_request(self, endpoint, params):
        """Basic request handler with retries"""
        retries = 5
        while retries > 0:
            try:
                response = requests.get(self.url + endpoint, params=params)
                if response.status_code != 200:
                    raise ValueError(f"Response {response.status_code}: {response.text}")
                
                result = response.json()
                if 'error' in result and result['error']['code'] == 10028:
                    wait_time = int(response.headers.get('retry-after', '2'))
                    time.sleep(wait_time)
                    retries -= 1
                    continue
                return result.get('result')
                
            except Exception as e:
                self.logger.error(f"Request failed: {e}")
                retries -= 1
                time.sleep(0.01)
        return None

    def get_instruments(self, currency, kind="option"):
        """Get list of available instruments"""
        result = self.__send_request("public/get_instruments", 
                                   {"currency": currency, "kind": kind, "expired": "false"})
        return [inst['instrument_name'] for inst in result] if result else []

    def get_orderbook(self, instrument, depth=10):
        """Get orderbook for single instrument"""
        return self.__send_request("public/get_order_book", 
                                 {"instrument_name": instrument, "depth": depth})

# Usage:
client = DeribitHTTP()
# 
# # Single collection:
# data = next(client.collect_data(['BTC'], interval_ms=5000))
#
# # Fixed runtime:
# for data in client.collect_data(['BTC'], interval_ms=5000, runtime_ms=60000):
#     process(data)
#
# # Continuous collection:
# for data in client.collect_continuous(['BTC'], interval_ms=5000, save_path='data'):
#     process(data)

In [27]:
import pandas as pd

deribithttp = DeribitHTTP()
instruments = deribithttp.get_instruments('BTC', kind='option')
instruments

['BTC-12JAN25-82000-C',
 'BTC-12JAN25-82000-P',
 'BTC-12JAN25-84000-C',
 'BTC-12JAN25-84000-P',
 'BTC-12JAN25-86000-C',
 'BTC-12JAN25-86000-P',
 'BTC-12JAN25-88000-C',
 'BTC-12JAN25-88000-P',
 'BTC-12JAN25-89000-C',
 'BTC-12JAN25-89000-P',
 'BTC-12JAN25-90000-C',
 'BTC-12JAN25-90000-P',
 'BTC-12JAN25-91000-C',
 'BTC-12JAN25-91000-P',
 'BTC-12JAN25-92000-C',
 'BTC-12JAN25-92000-P',
 'BTC-12JAN25-92500-C',
 'BTC-12JAN25-92500-P',
 'BTC-12JAN25-93000-C',
 'BTC-12JAN25-93000-P',
 'BTC-12JAN25-93500-C',
 'BTC-12JAN25-93500-P',
 'BTC-12JAN25-94000-C',
 'BTC-12JAN25-94000-P',
 'BTC-12JAN25-94500-C',
 'BTC-12JAN25-94500-P',
 'BTC-12JAN25-95000-C',
 'BTC-12JAN25-95000-P',
 'BTC-12JAN25-95500-C',
 'BTC-12JAN25-95500-P',
 'BTC-12JAN25-96000-C',
 'BTC-12JAN25-96000-P',
 'BTC-12JAN25-97000-C',
 'BTC-12JAN25-97000-P',
 'BTC-12JAN25-98000-C',
 'BTC-12JAN25-98000-P',
 'BTC-12JAN25-99000-C',
 'BTC-12JAN25-99000-P',
 'BTC-12JAN25-100000-C',
 'BTC-12JAN25-100000-P',
 'BTC-12JAN25-102000-C',
 'BTC-12JAN25

In [32]:
orderbook = deribithttp.get_orderbook(instruments[0])
orderbook

{'timestamp': 1736603341175,
 'state': 'open',
 'stats': {'high': None,
  'low': None,
  'price_change': None,
  'volume': 0.0,
  'volume_usd': 0.0},
 'greeks': {'delta': 0.99985,
  'gamma': 0.0,
  'vega': 0.02492,
  'theta': -1.07062,
  'rho': 1.7018},
 'change_id': 84659130221,
 'index_price': 94373.96,
 'instrument_name': 'BTC-12JAN25-82000-C',
 'bids': [[0.0955, 5.0], [0.014, 1.0]],
 'asks': [[0.165, 5.0]],
 'last_price': None,
 'settlement_price': 0.12991126,
 'min_price': 0.0955,
 'max_price': 0.1655,
 'open_interest': 0.0,
 'mark_price': 0.1313,
 'interest_rate': 0.0,
 'best_ask_price': 0.165,
 'best_bid_price': 0.0955,
 'mark_iv': 85.92,
 'bid_iv': 0.0,
 'ask_iv': 493.13,
 'underlying_price': 94393.0296,
 'underlying_index': 'SYN.BTC-12JAN25',
 'estimated_delivery_price': 94373.96,
 'best_ask_amount': 5.0,
 'best_bid_amount': 5.0}

In [18]:
data = client.fetch_all_data(
    currencies=['BTC'],
    timeframe='5m',
    kind='option',
    batch_size=40,
    max_workers=20,
    runtime_minutes=1
)
data

<generator object DeribitHTTP.fetch_all_data at 0x12656e5c0>

# 1 How do I get historical data of Deribit Orderbook?

1. Pick a specific instrument
2. Use the API to ping every second for get_order_book